# Building a Database for Crime Reports


The goal of this project build a database for storing data related to crimes that occurred in Boston.


The dataset is available in the file `Boston.csv` stored locally on my computer.


### Column Definitions

- **incident_number:** Represents the identifier of the crime
- **offense_code:** Identifier code for the committed crime
- **description:** Description of the crime
- **date:** Date when the crime happened 
- **day_of_the_week:** Corresponding day of the week
- **lat:** Location of the crime (latitude coordinates)
- **long:** Location of the crime (longitude coordinates)


### Recommendations

To get started, I recommend creating a database named `crimes_db` with a table – `boston_crimes` – using appropriate datatypes for storing the data from the `Boston.csv` file. The table will be created inside a schema named `crimes`. Additionally, create the `readonly` and `readwrite` groups with the appropriate privileges. Finally, create one user for each of these groups.

### Summary of Results:
I have been able to create a new database with it's own Schema. Database `crime_db` has two groups and 1 user in each group.





## Creating the Crime Database

In [1]:
import psycopg2

# Connecting to the main PostgreSQL database named "dq"
conn = psycopg2.connect(dbname="dq", user="dq")
conn.autocommit = True  # Set autocommit to True for immediate execution of SQL statements
cur = conn.cursor()     # Create a cursor for executing SQL statements

# Creating a new database named "crime_db"
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False  # Set autocommit to False for manual control over transactions

# Connecting to the newly created "crime_db" database
crime_db_conn = psycopg2.connect(dbname="crime_db", user="dq")
crime_cur = crime_db_conn.cursor()  # Create a cursor for the "crime_db" database

# Creating a schema named "crimes" within the "crime_db" database
crime_cur.execute("CREATE SCHEMA crimes")


## Obtaining the Column Names and Sample

Now we'll want to figure out the best datatype to use for our `crime_db` table.

We'll start by reading the column names from the `boston.csv` file as well as the first row. This way, we will have them at hand throughout this project so we can easily refer to them.

In [2]:
import csv

with open('boston.csv', 'r') as file:
    csv_reader = csv.reader(file)
    all_rows = list(csv_reader)
    col_headers = all_rows[0]
    first_row = all_rows[1]

# Now, col_headers contains the header row, and first_row contains the first data row
print("Header Row:", col_headers)
print("First Data Row:", first_row)


Header Row: ['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
First Data Row: ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Finding the Best Datatype to use for columns in my crime_db

#### Check number of rows to get the max value. This will determine the Storage size to use for the ID column.

In [3]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('boston.csv')
num_rows = len(df)
print(num_rows)

298329


We'll use 8 bytes for storing the values from the `incident_number` column because it's larger that 2 bytes which range only from -32768 to 32767

#### Check For Max value in the `offense_code` column.

In [4]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('boston.csv', header=None, skiprows=1)  # Assuming the header is not included in the CSV file

# Slice away the header for the dataset

# Convert values in index 1 to integers
index_1_values = df.iloc[:, 1].astype(int)

# Find the highest value in index 1
highest_value = index_1_values.max()

print("Highest value in index 1:", highest_value)

Highest value in index 1: 3831


We'll use 2 bytes for storing this values in our `crime_db`

#### Let's find out the distinct value in `day_of_the_week` column, as this will help us know if will be using an Enum datatype. There are only 7days in a week!

In [5]:
import csv

def get_col_set(csv_filename, col_index):
    column_values = set()

    with open(csv_filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)
        for row in reader:
            # Make sure the row has enough columns before trying to access col_index
            if col_index < len(row):
                column_values.add(row[col_index])

    return column_values


csv_filename = 'boston.csv'

for col_index in range(7):
    distinct_values = get_col_set(csv_filename, col_index)
    num_distinct_values = len(distinct_values)
    print("Column {}: {} distinct values".format(col_index, num_distinct_values))


Column 0: 298329 distinct values
Column 1: 219 distinct values
Column 2: 239 distinct values
Column 3: 1177 distinct values
Column 4: 7 distinct values
Column 5: 18177 distinct values
Column 6: 18177 distinct values


Columns with a low number of distinct values tend to be good candidates for enumerated datatype, which will be date_of_the_week. So the an enumerated datatype is what we'll use to store date of the week in our `crime_db`table. And moreover, they can only be 7days in a week.

#### Checking the Precision of the lat and Long column.

In [6]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('boston.csv')

# Check the column with index 5 or 6
column_index = 5
column_name = df.columns[column_index]

# Dictionary to store precision count for each element
precision_counts = {}

# Iterate through each element in the column
for value in df[column_name]:
    # Convert the value to string to handle decimal points
    value_str = str(value)
    # Find the position of the decimal point
    decimal_index = value_str.find('.')
    # Count the number of digits before and after the decimal point
    if decimal_index == -1:
        precision = len(value_str)
    else:
        precision = len(value_str[:decimal_index]) + len(value_str[decimal_index+1:])
    # Update the precision count for this precision
    if precision not in precision_counts:
        precision_counts[precision] = [value_str]
    else:
        precision_counts[precision].append(value_str)

# Print out precision counts
print("Different Precisions:")
for precision, values in precision_counts.items():
    print("Precision {}: {}".format(precision, len(values)))
    print("Sample Values:", values[:3])  # Print out sample values for each precision


Different Precisions:
Precision 16: 1491
Sample Values: ['42.27796370000001', '42.32960870000001', '42.29918970000001']
Precision 17: 9648
Sample Values: ['42.325694899999995', '42.347972399999996', '42.347972399999996']
Precision 6: 23
Sample Values: ['42.2812', '42.2812', '42.3349']
Precision 7: 495
Sample Values: ['42.37766', '42.37766', '42.28137']
Precision 8: 4941
Sample Values: ['42.351084', '42.351084', '42.351084']
Precision 9: 17262
Sample Values: ['42.3503876', '42.3218854', '42.3353997']
Precision 10: 264469
Sample Values: ['42.35779134', '42.30682138', '42.34658879']


We'll use Decimal type for storing this. Precision will be 17 and scale will be 15 for both long and lat column in our `crime_db`

**NOTE:** FOR Description and Date column, We'll use text and datetime type. Reason is that the values in description column have uncertain cizes and the datetime column already has the format that the datetime type support

## Creating the Table & Loading Data into it.

In [7]:
#Before that we'' create the Enum Type first

crime_cur.execute("CREATE TYPE week_days AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');")

# Define the SQL statement to create the table
create_table_sql = """
                   CREATE TABLE crimes.boston_crimes (
                       incident_number  BIGINT,
                       offense_code SMALLINT,
                       description TEXT,
                       date DATE,
                       day_of_the_week week_days,
                       lat DECIMAL(17, 15),
                       long DECIMAL(17, 15)
                   );
                   """

crime_cur.execute(create_table_sql)

#Load data into the `bostom_crimes` Table
import csv
with open("boston.csv", "r") as f:
    crime_cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

# Revoking Public Privileges
crime_cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
crime_cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating User Groups & Users

We've made sure that we aren't going to inadvertently inherit privileges from the public group. The next step is to create our two user groups.

In [8]:
# Created two groups named readonly and readwrite with NOLOGIN option
crime_cur.execute("CREATE GROUP readonly NOLOGIN;")
crime_cur.execute("CREATE GROUP readwrite NOLOGIN;")

# Grant CONNECT to the crime_db to both groups
crime_cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
crime_cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

# Grant USAGE to the crimes schema to both groups
crime_cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
crime_cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

# Granted group-specific privileges to each group on all tables in the crimes schema
# readonly group should only be granted the SELECT privilege
crime_cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

# readwrite group should be granted the SELECT, INSERT, DELETE, and UPDATE privileges
crime_cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

# Create a user named data_analyst with password secret1
crime_cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")

# Assign data_analyst user to the readonly group
crime_cur.execute("GRANT readonly TO data_analyst;")

# Create a user named data_scientist with password secret2
crime_cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")

# Assign data_scientist user to the readwrite group
crime_cur.execute("GRANT readwrite TO data_scientist;")